In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import graph_tool.all as gt
import sys
# sys.insert_path('/Users/glover.co/Documents/laszlo/NetDesign/scripts/')
import assembly_tree as at
import mcmc
import treelib
from itertools import product

In [2]:
X = np.vstack([np.eye(3) for i in range(2)])
O = np.array([[0,1,1],[1,0,1],[1,1,2]])
capacity = O.sum(axis=1,dtype=int)
target = nx.Graph()
target.add_nodes_from(np.arange(6))
target.add_edges_from([[0,1],[1,2],[2,0],[3,4],[4,5],[5,3],[2,5]])

In [3]:
ex0 = mcmc.AssemblyTree(target,X,O,capacity)

(array([1, 2, 7]), [<networkx.classes.graph.Graph object at 0x168db1400>, <networkx.classes.graph.Graph object at 0x30e2781b0>, <networkx.classes.graph.Graph object at 0x30e2522d0>], array([2, 1, 0]))


In [4]:
ex0.Tree.get_node(0).data.p

array([1.])

In [5]:
ex0.update_tree(dist=[1,0,0,0])

[Node(tag=1, identifier=1, data=<mcmc.AssemblyNode object at 0x30e1fa5d0>), Node(tag=2, identifier=2, data=<mcmc.AssemblyNode object at 0x30e1fa850>), Node(tag=3, identifier=3, data=<mcmc.AssemblyNode object at 0x11b160b00>)]
Node(tag=1, identifier=1, data=<mcmc.AssemblyNode object at 0x30e1fa5d0>)
(array([8, 2]), [<networkx.classes.graph.Graph object at 0x30e252cd0>, <networkx.classes.graph.Graph object at 0x30e2d08d0>], array([0, 1]))
Node(tag=2, identifier=2, data=<mcmc.AssemblyNode object at 0x30e1fa850>)
(array([10]), [<networkx.classes.graph.Graph object at 0x30e2d1750>], array([0]))
Node(tag=3, identifier=3, data=<mcmc.AssemblyNode object at 0x11b160b00>)
(array([10]), [<networkx.classes.graph.Graph object at 0x30e2522d0>], array([0]))
Node(tag=0, identifier=0, data=<mcmc.AssemblyNode object at 0x30e235a90>)
PROBS [(np.float64(0.4), np.float64(1.0), np.float64(1.0)), (np.float64(0.6), np.float64(1.0), np.float64(1.0))]
node 0
(array([1, 1, 1, 4, 3]), [<networkx.classes.graph.Gra

TypeError: only integer scalar arrays can be converted to a scalar index

In [7]:
x = np.array([1])
np.argsort(x)[::-1]

array([0])

In [ ]:
subgraph_combinations = list(product(*subgraphs))


In [ ]:
subgraph_combinations
for i, subgraph in enumerate(subgraph_combinations):
    initial_graph = nx.Graph()
    for s in subgraph:
        initial_graph = nx.compose(initial_graph, s)

In [ ]:
initial_graph = nx.Graph()
initial_graph.add_nodes_from([np.int64(4),np.int64(0)])

In [25]:
sub_nodes = ex0.Tree.get_node(1).data.nodes

[np.int64(3), np.int64(2)]

In [30]:
p, samples, idx = at.prob_dist(ex0.X[sub_nodes,:],ex0.O,ex0.capacity,initial_graph=None,max_edges=True,labeled=True)

In [32]:
true_A = ex0.target_A[:,sub_nodes][sub_nodes,:]
cur_a = nx.adjacency_matrix(samples[0]).todense()
np.allclose(true_A,cur_a)

False

In [34]:
true_A, cur_a

(array([[0, 0],
        [0, 0]]),
 array([[0, 1],
        [1, 0]]))

In [ ]:
children = ex0.Tree.get_node(0).successors(ex0.Tree.identifier)

In [56]:
zero_prob = np.prod(np.array([np.prod(x) for c in children for x in ex0.Tree.get_node(c).data.logP ]))

In [63]:
subgraph_list = [ex0.Tree.get_node(c).data.subgraph for c in children]

In [64]:
subgraph_list

[[],
 [],

In [9]:
x = 1
y = 2
z = 3
true_list = [x,y,z]
for j, s in enumerate(true_list):
    true_list[j] = 4

In [70]:
list1 = [1,2,3,4,5,6]
list2 = [4,5,6,7]
list3 = [2,1]

In [71]:
from itertools import product

In [74]:
np.prod(list(product(list1, list2, list3)),axis=1)

array([ 8,  4, 10,  5, 12,  6, 14,  7, 16,  8, 20, 10, 24, 12, 28, 14, 24,
       12, 30, 15, 36, 18, 42, 21, 32, 16, 40, 20, 48, 24, 56, 28, 40, 20,
       50, 25, 60, 30, 70, 35, 48, 24, 60, 30, 72, 36, 84, 42])